In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel

# ----------------------------
# 1. Charger modèle LoRA + tokenizer
# ----------------------------
output_dir = r"C:\Users\HP\desktop\NMAP_AI\backend\Agents\Agent_medium\T5"
tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(output_dir, device_map="auto")
model = PeftModel.from_pretrained(model, output_dir)

# ----------------------------
# 2. Forcer le modèle sur GPU
# ----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()  # mode évaluation

# ----------------------------
# 3. Fonction pour générer du texte
# ----------------------------
def ask(prompt: str, max_new_tokens: int = 128):
    # Tokenize et envoyer les tenseurs sur GPU
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    # Génération
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    # Décodage
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ----------------------------
# 4. Test du modèle
# ----------------------------
question = "Scan all ports on 192.168.1.20"
answer = ask(question)
print("Prompt :", question)
print("Generated command:", answer)


Prompt : Scan all ports on 192.168.1.20
Generated command: nmap -p- 192.168.1.20


In [8]:
tests = [
    "Scan all ports on 192.168.1.20",
    "Run default and vuln scripts on 10.0.0.1",
    "Scan specific ports 22,80,443 on 172.16.0.5",
    "Detect OS and service versions on target.com",
    "Fast scan with aggressive timing on 10.10.10.10",
    "Stealth SYN scan on 192.168.1.0/24",
    "check anonymous ftp access",
    "Scan top ports with version detection and OS detection on 192.168.1.50",
    "Scan top ports with version detection and OS detection on 192.168.1.50",
    "Quick scan with default scripts and version detection on 10.0.5.20"
]

for q in tests:
    print("Prompt :", q)
    print("Generated command:", ask(q))
    print("------------------------------------------------")


Prompt : Scan all ports on 192.168.1.20
Generated command: nmap -p- 192.168.1.20
------------------------------------------------
Prompt : Run default and vuln scripts on 10.0.0.1
Generated command: nmap -p- 10.0.0.1 --script default --script vuln target
------------------------------------------------
Prompt : Scan specific ports 22,80,443 on 172.16.0.5
Generated command: nmap -p 22,80,443 172.16.0.5
------------------------------------------------
Prompt : Detect OS and service versions on target.com
Generated command: nmap -O --version-version target
------------------------------------------------
Prompt : Fast scan with aggressive timing on 10.10.10.10
Generated command: nmap -sA -T5 10.10.10.10
------------------------------------------------
Prompt : Stealth SYN scan on 192.168.1.0/24
Generated command: nmap -sS 192.168.1.0/24
------------------------------------------------
Prompt : check anonymous ftp access
Generated command: nmap -p- anonymous ftp
---------------------------